# 5输出解析器

语言模型输出文本。但很多时候，您可能希望获得更结构化的信息，而不仅仅是文本回复。这就是输出解析器的用武之地。


输出分析器是帮助构建语言模型响应的类。输出分析器必须实现两种主要方法：

“获取格式指令”：返回一个字符串的方法，其中包含有关如何格式化语言模型输出的说明。

“Parse”：一种接收字符串（假设是来自语言模型的响应）并将其解析为某种结构的方法。

然后是一个可选的：

“Parse with prompt”：一种方法，它接受一个字符串（假设是来自语言模型的响应）和一个提示（假设是生成此类响应的提示）并将其解析为某种结构。提示主要在 OutputParser 想要以某种方式重试或修复输出时提供，并且需要来自提示的信息才能执行此操作。

In [4]:
#这段代码的主要目的是使用一个预训练的语言模型从OpenAI来生成并验证一个笑话。
# 导入必要的模块和类
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# 定义模型名称和温度（影响模型的随机性）
model_name = 'text-davinci-003'
temperature = 0.0

# 初始化OpenAI模型
model = OpenAI(model_name=model_name, temperature=temperature)

# 定义想要的数据结构，这里是一个笑话的结构，包含设置和冷笑话
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")  # 笑话的设置部分
    punchline: str = Field(description="answer to resolve the joke")  # 笑话的冷笑话部分

    # 使用Pydantic添加自定义验证逻辑，确保设置部分以问号结束
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '？':
            raise ValueError("Badly formed question!")
        return field

# 设置一个解析器，并将指令注入到提示模板中
parser = PydanticOutputParser(pydantic_object=Joke)

# 定义提示模板
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 定义一个查询，目的是提示语言模型填充上述数据结构
joke_query = "给我用中文讲个笑话."

# 格式化提示
_input = prompt.format_prompt(query=joke_query)

# 使用模型生成输出
output = model(_input.to_string())

# 使用解析器解析输出
parser.parse(output)


Joke(setup='为什么有的人喜欢吃苹果？', punchline='因为它们是甜甜的！')

## 5.1 列表解析器
当您想要返回逗号分隔项的列表时，可以使用此输出分析器。

In [6]:
# 导入必要的模块和类
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# 初始化一个解析由逗号分隔的列表的解析器
output_parser = CommaSeparatedListOutputParser()

# 获取解析器的格式指令
format_instructions = output_parser.get_format_instructions()

# 定义提示模板，用于生成关于特定主题的由逗号分隔的列表
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# 初始化OpenAI模型，温度设置为0（生成的回答将更加确定，少有随机性）
model = OpenAI(temperature=0)

# 格式化提示，这里的主题是“ice cream flavors”（冰淇淋口味）
_input = prompt.format(subject="冰淇淋口味")

# 使用模型生成输出
output = model(_input)

# 使用解析器解析输出
output_parser.parse(output)


['草莓', '香草', '抹茶', '巧克力', '杏仁酥']

## 5.2 日期时间解析器

此输出解析器显示将LLM输出解析为日期时间格式。

In [10]:
# 导入必要的模块和类
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# 初始化一个日期时间输出解析器
output_parser = DatetimeOutputParser()

# 定义提示模板，用于引导模型回答用户的问题
template = """回答用户的问题:

{question}

{format_instructions}"""

# 使用模板创建一个提示实例
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# 初始化一个LLMChain，它结合了提示和OpenAI模型来生成输出
chain = LLMChain(prompt=prompt, llm=OpenAI())

# 运行链来获取关于特定问题的答案，这里的问题是“bitcoin是什么时候成立的？”
output = chain.run("bitcoin是什么时候成立的？用英文格式输出时间")


In [11]:
output

'\n\n2009-01-03T18:15:05.000000Z'

In [12]:
output_parser.parse(output)

datetime.datetime(2009, 1, 3, 18, 15, 5)

## 5.3 枚举解析器

In [13]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum


class Colors(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

parser = EnumOutputParser(enum=Colors)


In [14]:
parser.parse("red")

<Colors.RED: 'red'>

In [15]:
# Can handle spaces
parser.parse(" green")

<Colors.GREEN: 'green'>

In [19]:
# And new lines
parser.parse("blue\n")

<Colors.BLUE: 'blue'>

In [17]:
# And raises errors when appropriate
parser.parse("yellow")

OutputParserException: Response 'yellow' is not one of the expected values: ['red', 'green', 'blue']

## 5.4 自动修复解析器
此输出解析器包装另一个输出解析器，如果第一个输出解析器失败，它会调用另一个 LLM 以修复任何错误。

但是除了抛出错误之外，我们还可以做其他事情。具体来说，我们可以将格式错误的输出以及格式化的指令传递给模型，并要求它修复它。

In [20]:
# 导入所需的库和模块
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# 定义一个表示演员的数据结构，包括他们的名字和他们出演的电影列表
class Actor(BaseModel):
    name: str = Field(description="name of an actor")                   # 演员的名字
    film_names: List[str] = Field(description="list of names of films they starred in")  # 他们出演的电影列表

# 定义一个查询，用于提示生成随机演员的电影作品列表
actor_query = "Generate the filmography for a random actor."

# 使用`Actor`模型初始化解析器
parser = PydanticOutputParser(pydantic_object=Actor)

# 定义一个格式错误的字符串数据
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

# 使用解析器尝试解析上述数据
parser.parse(misformatted)


OutputParserException: Failed to parse Actor from completion {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}. Got: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [23]:
# 要使misformatted字符串正确格式化，您需要确保它是一个有效的JSON字符串。这意味着您应该使用双引号（"）而不是单引号（'）来包围键和值。以下是修改后的
parser.parse('{"name": "Tom Hanks", "film_names": ["Forrest Gump"]}')

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

In [21]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

## 5.5 Pydantic解析器
此输出分析器允许用户指定任意 JSON 架构，并在 LLM 中查询符合该架构的 JSON 输出。

请记住，大型语言模型是有缺陷的抽象！您需要使用具备足够容量以生成格式良好的JSON的LLM。在OpenAI系列中，DaVinci可以可靠地完成此任务，但Curie的能力已经显著下降。

In [24]:
# 导入必要的模块和类
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# 定义模型名称和温度（影响模型的随机性）
model_name = 'text-davinci-003'
temperature = 0.0

# 初始化OpenAI模型
model = OpenAI(model_name=model_name, temperature=temperature)

# 定义想要的数据结构，这里是一个笑话的结构，包含设置和冷笑话
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")  # 笑话的设置部分
    punchline: str = Field(description="answer to resolve the joke")  # 笑话的冷笑话部分

    # 使用Pydantic添加自定义验证逻辑，确保设置部分以问号结束
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

# 定义一个查询，目的是提示语言模型填充上述数据结构
joke_query = "Tell me a joke."

# 设置一个解析器，并将指令注入到提示模板中
parser = PydanticOutputParser(pydantic_object=Joke)

# 定义提示模板
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 格式化提示
_input = prompt.format_prompt(query=joke_query)

# 使用模型生成输出
output = model(_input.to_string())

# 使用解析器解析输出
parser.parse(output)


Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [25]:
# 导入必要的模块和类
from pydantic import BaseModel, Field
from typing import List

# 定义一个表示演员及其电影作品列表的数据结构
class Actor(BaseModel):
    name: str = Field(description="name of an actor")                   # 演员的名字
    film_names: List[str] = Field(description="list of names of films they starred in")  # 他们出演的电影列表

# 定义一个查询，用于提示生成随机演员的电影作品列表
actor_query = "Generate the filmography for a random actor."

# 使用`Actor`模型初始化解析器
parser = PydanticOutputParser(pydantic_object=Actor)

# 定义提示模板，用于引导模型生成关于特定查询的答案
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 格式化提示
_input = prompt.format_prompt(query=actor_query)

# 使用模型生成输出
output = model(_input.to_string())

# 使用解析器解析输出
parser.parse(output)


Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Saving Private Ryan', 'The Green Mile', 'Cast Away', 'Toy Story'])

## 5.6 重试解析器
虽然在某些情况下，可以通过仅查看输出来修复任何解析错误，但在其他情况下则不能。

例如，当输出不仅格式不正确，而且部分完成时。请考虑以下示例。

In [30]:
# 导入必要的模块和类
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# 定义提示模板，用于引导模型根据用户的问题提供相应的操作和操作输入
template = """基于用户的问题，提供应采取的操作及操作的输入。
{format_instructions}
问题: {query}
回答:"""

# 定义一个表示操作及其输入的数据结构
class Action(BaseModel):
    action: str = Field(description="要采取的操作")
    action_input: str = Field(description="操作的输入")

# 使用`Action`模型初始化解析器
parser = PydanticOutputParser(pydantic_object=Action)

# 定义提示模板
prompt = PromptTemplate(
    template="回答用户的问题。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 格式化提示
prompt_value = prompt.format_prompt(query="谁是马云的女朋友？")

# 定义一个不完整的回应示例
bad_response = '{"action": "search"}'


In [31]:
#如果我们尝试按原样解析此响应，我们将收到错误
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  field required (type=value_error.missing)

In [32]:
# 如果我们尝试使用 来 OutputFixingParser 修复此错误，它会感到困惑 - 即，它不知道实际为操作输入放置什么。
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
fix_parser.parse(bad_response)

Action(action='search', action_input='query')

In [33]:
# 但是，如果我们使用 RetryOutputParser ，它将尝试使用提示和模型来修复错误。
from langchain.output_parsers import RetryWithErrorOutputParser
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0)
)
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='search', action_input='马云女朋友')

## 5.7 结构化输出解析器
当您想要返回多个字段时，可以使用此输出分析器。

虽然 Pydantic/JSON 解析器功能更强大，但我们最初尝试使用仅包含文本字段的数据结构。

In [34]:
# 导入必要的模块和类
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# 定义响应模式，用于指导模型提供特定类型的输出
response_schemas = [
    ResponseSchema(name="answer", description="回答用户的问题"),
    ResponseSchema(name="source", description="回答用户问题的来源，应为一个网站。")
]

# 使用响应模式初始化一个结构化输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 获取解析器的格式指令
format_instructions = output_parser.get_format_instructions()

# 定义提示模板，用于引导模型根据用户的问题提供答案及答案的来源
prompt = PromptTemplate(
    template="尽可能回答用户的问题。\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

# 初始化OpenAI模型，温度设置为0（生成的回答将更加确定，少有随机性）
model = OpenAI(temperature=0)

# 格式化提示，这里的问题是“法国的首都是什么？”
_input = prompt.format_prompt(question="法国的首都是什么？")

# 使用模型生成输出
output = model(_input.to_string())

# 使用解析器解析输出
output_parser.parse(output)


{'answer': '巴黎',
 'source': 'https://baike.baidu.com/item/%E6%B3%95%E5%9B%BD/1045'}

In [35]:
# 导入必要的模块和类
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

# 初始化聊天模型，温度设置为0（生成的回答将更加确定，少有随机性）
chat_model = ChatOpenAI(temperature=0)

# 定义聊天提示模板，该模板包含一系列的消息对象
# 这里我们只定义一个消息，该消息提示模型尽可能地回答用户的问题
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("尽可能回答用户的问题。\n{format_instructions}\n{question}")
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

# 格式化提示，这里的问题是“法国的首都是什么？”
_input = prompt.format_prompt(question="法国的首都是什么？")

# 使用聊天模型生成输出
output = chat_model(_input.to_messages())

# 使用解析器解析输出的内容
output_parser.parse(output.content)


{'answer': '法国的首都是巴黎。',
 'source': 'https://zh.wikipedia.org/wiki/%E5%B7%B4%E9%BB%8E'}